In [1]:
%pip install google-maps-places


   ---------------------------------------- 0.0/74.6 kB ? eta -:--:--
   ---------------------------------------- 74.6/74.6 kB 4.3 MB/s eta 0:00:00


In [2]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [45]:
import os
from dotenv import load_dotenv
load_dotenv()
maps_key = os.getenv('GOOGLE_MAPS_API')

In [7]:
import geocoder
def get_current_gps_coordinates():
    g = geocoder.ip('me')#this function is used to find the current information using our IP Add
    if g.latlng is not None: #g.latlng tells if the coordinates are found or not
        return g.latlng
    else:
        return None

In [12]:
def construct_budget(budget):
    price_dict = {'casual': 'PRICE_LEVEL_INEXPENSIVE', 
                  'mid-range': 'PRICE_LEVEL_MODERATE', 
                  'fine dining': ['PRICE_LEVEL_EXPENSIVE', 
                                  'PRICE_LEVEL_VERY_EXPENSIVE']
                }
    return price_dict[budget]

In [10]:
import requests
def get_maps_coordinates():
    url = f'https://www.googleapis.com/geolocation/v1/geolocate?key={maps_key}'

    coordinates = requests.post(url).json()
    # return a tuple with latitude and longitude
    return (coordinates['location']['lat'], coordinates['location']['lng'])

In [23]:
%pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib


  Using cached google_api_python_client-2.134.0-py2.py3-none-any.whl.metadata (6.7 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached google_auth_oauthlib-1.2.0-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
Using cached google_api_python_client-2.134.0-py2.py3-none-any.whl (11.9 MB)
Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl (9.3 kB)
Using cached google_auth_oauthlib-1.2.0-py2.py3-none-any.whl (24 kB)
Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl (24 kB)
Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)
   ---------------------------------------- 0.0/151.7 kB ? eta -:--:--
   ---------------------------------------- 151.7/151.7 kB 8.8 MB/s 

In [1]:
import os
import datetime
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request

def create_service(client_secret_file, api_name, api_version, *scopes, prefix=''):
    CLIENT_SECRET_FILE = client_secret_file
    API_SERVICE_NAME = api_name
    API_VERSION = api_version
    SCOPES = [scope for scope in scopes[0]]
    
    creds = None
    working_dir = os.getcwd()
    token_dir = 'token_files'
    token_file = f'token_{API_SERVICE_NAME}_{API_VERSION}{prefix}.json'

    ### Check if token dir exists first, if not, create the folder
    if not os.path.exists(os.path.join(working_dir, token_dir)):
        os.mkdir(os.path.join(working_dir, token_dir))

    if os.path.exists(os.path.join(working_dir, token_dir, token_file)):
        creds = Credentials.from_authorized_user_file(os.path.join(working_dir, token_dir, token_file), SCOPES)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=0)

        with open(os.path.join(working_dir, token_dir, token_file), 'w') as token:
            token.write(creds.to_json())

    try:
        service = build(API_SERVICE_NAME, API_VERSION, credentials=creds, static_discovery=False)
        print(API_SERVICE_NAME, API_VERSION, 'service created successfully')
        return service
    except Exception as e:
        print(e)
        print(f'Failed to create service instance for {API_SERVICE_NAME}')
        os.remove(os.path.join(working_dir, token_dir, token_file))
        return None

def convert_to_RFC_datetime(year=1900, month=1, day=1, hour=0, minute=0):
    dt = datetime.datetime(year, month, day, hour, minute, 0).isoformat() + 'Z'
    return dt

In [36]:
def text_search(query, budget, num_recs, latitude, longitude):
    request_body = {
        'textQuery': query,
        'maxResultCount': num_recs,
        'locationBias': {
            'circle': {
                'center': {
                    'latitude': latitude,
                    'longitude': longitude,
                },
                'radius': 1000
            }
        },
        'rankPreference': "RELEVANCE",
        'priceLevels': construct_budget(budget),
    }
    return request_body

In [37]:
def places_hub(places_type, query, budget, num_recs):
    client_secret_file = 'client_secret_desktop.json'
    API_NAME = 'places'
    API_VERSION = 'v1'
    SCOPES = ['https://www.googleapis.com/auth/cloud-platform']
    service = create_service(client_secret_file, API_NAME, API_VERSION, SCOPES)
    
    coordinates = get_maps_coordinates()
    try:
        assert coordinates is not None, "coordinates weren't retrieved"
        latitude, longitude = coordinates
    except Exception as e:
        print(e)
    
    if places_type == 'text_search':
        request_body = text_search(query, budget, num_recs, latitude, longitude )
        response = service.places().searchText(
        body = request_body,
        # no spaces in the fields parameter!!!
        fields = 'places.displayName,places.primaryType,places.rating,places.userRatingCount,places.priceLevel'
        ).execute()
        
    elif places_type == 'nearby_search':
        request_body = 'pending'
        
    print(response)
    return response['places']

In [42]:
places_hub('text_search', 'ceviche', 'mid-range', 5)

places v1 service created successfully
{'places': [{'rating': 4.2, 'priceLevel': 'PRICE_LEVEL_MODERATE', 'userRatingCount': 471, 'displayName': {'text': 'Limón', 'languageCode': 'en'}, 'primaryType': 'restaurant'}, {'rating': 4.7, 'priceLevel': 'PRICE_LEVEL_MODERATE', 'userRatingCount': 88, 'displayName': {'text': "Emelina's 2", 'languageCode': 'en'}, 'primaryType': 'restaurant'}, {'rating': 4.4, 'priceLevel': 'PRICE_LEVEL_MODERATE', 'userRatingCount': 249, 'displayName': {'text': 'Pacific Catch', 'languageCode': 'en'}, 'primaryType': 'seafood_restaurant'}, {'rating': 4.7, 'priceLevel': 'PRICE_LEVEL_MODERATE', 'userRatingCount': 105, 'displayName': {'text': 'Inka’s restaurant', 'languageCode': 'en'}, 'primaryType': 'restaurant'}, {'rating': 4.6, 'priceLevel': 'PRICE_LEVEL_MODERATE', 'userRatingCount': 1572, 'displayName': {'text': 'Mariscos Costa Alegre', 'languageCode': 'en'}, 'primaryType': 'mexican_restaurant'}]}


[{'rating': 4.2,
  'priceLevel': 'PRICE_LEVEL_MODERATE',
  'userRatingCount': 471,
  'displayName': {'text': 'Limón', 'languageCode': 'en'},
  'primaryType': 'restaurant'},
 {'rating': 4.7,
  'priceLevel': 'PRICE_LEVEL_MODERATE',
  'userRatingCount': 88,
  'displayName': {'text': "Emelina's 2", 'languageCode': 'en'},
  'primaryType': 'restaurant'},
 {'rating': 4.4,
  'priceLevel': 'PRICE_LEVEL_MODERATE',
  'userRatingCount': 249,
  'displayName': {'text': 'Pacific Catch', 'languageCode': 'en'},
  'primaryType': 'seafood_restaurant'},
 {'rating': 4.7,
  'priceLevel': 'PRICE_LEVEL_MODERATE',
  'userRatingCount': 105,
  'displayName': {'text': 'Inka’s restaurant', 'languageCode': 'en'},
  'primaryType': 'restaurant'},
 {'rating': 4.6,
  'priceLevel': 'PRICE_LEVEL_MODERATE',
  'userRatingCount': 1572,
  'displayName': {'text': 'Mariscos Costa Alegre', 'languageCode': 'en'},
  'primaryType': 'mexican_restaurant'}]